In [ ]:
import torch
from torch import nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용할 장치: {device}")

print("데이터를 로드하는 중...")
df = pd.read_csv("item.csv")

def combine_text(row):
    return f"{row['title']} [SEP] {row['short_description']}"

print("제목과 설명을 하나의 텍스트로 결합하는 중...")
df['combined_text'] = df.apply(combine_text, axis=1)

# 차원 축소를 위한 선형 레이어 추가
class BertWithProjection(nn.Module):
    def __init__(self, output_dim=256):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.projection = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled = outputs.pooler_output
        return self.projection(pooled)

print("BERT 모델과 토크나이저를 로드하는 중...")
bert_model = BertWithProjection(output_dim=256).to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def get_combined_embedding(text, tokenizer, model, max_length=512):
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        padding=True,
        truncation=True, 
        max_length=max_length
    ).to(device)
    
    with torch.no_grad():
        embedding = model(**inputs)
    return embedding.squeeze().cpu().numpy()

print("설명+제목 텍스트 임베딩 생성 중...")
df['combined_embedding'] = [
    get_combined_embedding(text, tokenizer, bert_model) 
    for text in tqdm(df['combined_text'], desc="설명+제목 임베딩 생성")
]

print("태그와 카테고리를 정수 인코딩하는 중...")
tag_encoder = LabelEncoder()
category_encoder = LabelEncoder()

df['encoded_tags'] = tag_encoder.fit_transform(df['tags_sum'])
df['encoded_categories'] = category_encoder.fit_transform(df['categories'])

class EmbeddingLayer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
    
    def forward(self, x):
        return self.embedding(x)

print("태그와 카테고리 임베딩 레이어를 초기화하는 중...")
tag_embedding_layer = EmbeddingLayer(num_embeddings=len(tag_encoder.classes_), embedding_dim=64).to(device)
category_embedding_layer = EmbeddingLayer(num_embeddings=len(category_encoder.classes_), embedding_dim=32).to(device)
print("len(tag_encoder.classes_): ", len(tag_encoder.classes_))
print("len(category_encoder.classes_): ", len(category_encoder.classes_))

# 태그와 카테고리 임베딩 계산
print("태그와 카테고리 임베딩을 계산하는 중...")
tag_embeddings = tag_embedding_layer(torch.tensor(df['encoded_tags'].values).to(device))
category_embeddings = category_embedding_layer(torch.tensor(df['encoded_categories'].values).to(device))

# item_id와 임베딩 매핑 생성
print("item_id와 임베딩 매핑 생성...")
embeddings_df = pd.DataFrame({
    'item_id': df['item_id'].values,
    'tag_embedding': list(tag_embeddings.detach().cpu().numpy()),
    'category_embedding': list(category_embeddings.detach().cpu().numpy())
})

print("최종 임베딩이 완료되었습니다!")
print(f"최종 임베딩 shape: {embeddings_df.shape}")
print("\n결과 예시:")
print(embeddings_df.head())

# 최종 임베딩 저장
print("최종 임베딩을 저장하는 중...")
embeddings_df.to_csv("item_embeddings.csv", index=False)
embeddings_df.to_pickle("item_embeddings.pkl")

사용할 장치: cuda
데이터를 로드하는 중...
BERT 모델과 토크나이저를 로드하는 중...
제목과 설명을 BERT로 임베딩하는 중...


제목 임베딩: 100%|██████████| 37141/37141 [04:35<00:00, 134.86it/s]


태그와 카테고리를 정수 인코딩하는 중...
태그와 카테고리 임베딩 레이어를 초기화하는 중...
태그와 카테고리 임베딩을 계산하는 중...
모든 임베딩을 결합하는 중...
최종 임베딩이 완료되었습니다!
최종 임베딩 shape: (37141, 1632)
